In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

### Read image into standard 2-d numpy array

In [ ]:
data = fits.getdata('image.fits')

### Show the original image

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('original-image.png')

### Background work

Measure a spatially varying background on the image

In [ ]:
bkg = sep.Background(data)

Evaluate background as 2-d array, same size as original image

In [ ]:
bkg_image = bkg.back()

Show the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('original-background-image.png')

Evaluate the background noise as a 2-d array, same size as original image

In [ ]:
bkg_rms = bkg.rms()

Show the background noise

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('original-background-noise.png')

Subtract the background

In [ ]:
data_sub = data - bkg

### Detecting objects in image

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

Plotting where each object is detected in the image

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
               width=6*objects['a'][i],
               height=6*objects['b'][i],
               angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig('original-detected-objects.png')

### Obtaining fluxes from objects in image

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                    3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))